In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
from scipy.misc import derivative
import soundfile as sf

In [2]:
def read_wav_file(file_path):
    """
    Đọc tệp WAV và trả về dữ liệu âm thanh (y) và tốc độ lấy mẫu (fs).
    """
    y, fs = librosa.load(file_path)
    return y, fs

In [3]:
def calculate_ste(y, frame_size=256, hop_size=128):
    """
    Tính toán năng lượng theo thời gian ngắn (STE) của tín hiệu âm thanh.

    Parameters:
        y (numpy.ndarray): Tín hiệu âm thanh.
        frame_size (int): Kích thước của khung.
        hop_size (int): Kích thước của bước nhảy.

    Returns:
        numpy.ndarray: Mảng STE.
    """
    num_frames = len(y) - frame_size + hop_size
    num_frames = num_frames // hop_size
    ste = np.zeros(num_frames)
    
    for i in range(num_frames):
        start = i * hop_size
        end = start + frame_size
        ste[i] = np.sum(np.abs(y[start:end]) ** 2)
    
    return ste

In [4]:
def normalize(ste):
    """
    Chuẩn hóa năng lượng theo thời gian ngắn (STE).

    Parameters:
        ste (numpy.ndarray): Mảng STE.

    Returns:
        numpy.ndarray: Mảng STE đã được chuẩn hóa.
    """
    return ste / np.max(ste)

In [5]:
def find_voiced_segments(ste, threshold):
    """
    Tìm các phân đoạn nói dựa trên giá trị STE và ngưỡng tốt nhất.
    """
    starts = []
    ends = []
    start = None
    end = None

    for i in range(len(ste)):
        if ste[i] > threshold:
            if end is None:
                start = i
            end = i + 1
        else:
            if start is not None:
                starts.append(start)
                ends.append(end)
                start = None
                end = None

    return starts, ends


In [6]:
def remove_short_silences(starts, ends, hop_size, fs, min_silence_duration=0.3):
    """
    Loại bỏ các khoảng lặng ngắn.
    """
    i = 0
    while i < (len(ends) - 1):
        if (starts[i + 1] * hop_size / fs) - (ends[i] * hop_size / fs) <= min_silence_duration:
            starts.pop(i + 1)
            ends.pop(i)
        else:
            i += 1
    
    return starts, ends

In [7]:
def create_output_folder(output_folder):
    """
    Kiểm tra và tạo thư mục đầu ra nếu nó không tồn tại.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

In [8]:
def create_output_folder(output_folder):
    """
    Kiểm tra và tạo thư mục đầu ra nếu nó không tồn tại.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

In [9]:
def calculate_sample_indices(start_time, end_time, sr):
    """
    Tính chỉ số mẫu bắt đầu và kết thúc từ thời gian (giây).
    """
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    return start_sample, end_sample

In [10]:
def cut_audio_segment(y, start_sample, end_sample):
    """
    Cắt đoạn âm thanh từ mẫu bắt đầu đến mẫu kết thúc.
    """
    segment = y[start_sample:end_sample]
    return segment

In [11]:
def generate_output_file_name(input_file, start_time, end_time):
    """
    Tạo tên file cho đoạn cắt.
    """
    file_name = os.path.basename(input_file)
    file_name_without_extension = os.path.splitext(file_name)[0]
    output_file_name = f"{file_name_without_extension}_{start_time}s_to_{end_time}s.wav"
    return output_file_name

In [12]:
def save_audio_segment(segment, output_file_path, sr):
    """
    Lưu đoạn âm thanh vào tệp WAV.
    """
    print(output_file_path)
    sf.write(output_file_path, segment, sr)

In [13]:
def cut_and_save_segment(input_file, output_folder, start_time, end_time):
    """
    Cắt một đoạn từ tệp âm thanh đầu vào từ giây a đến giây b và lưu vào thư mục đầu ra.
    """
    create_output_folder(output_folder)

    y, sr = librosa.load(input_file, sr=None)
    start_sample, end_sample = calculate_sample_indices(start_time, end_time, sr)
    segment = cut_audio_segment(y, start_sample, end_sample)
    output_file_name = generate_output_file_name(input_file, start_time, end_time)
    output_file_path = os.path.join(output_folder, output_file_name)
    save_audio_segment(segment, output_file_path, sr)
    print(start_time, end_time)

In [14]:
def process_ste(y, fs, frame_size, hop_size, best_threshold, min_silence_duration):
    """
    Xử lý Short-term energy (STE) và loại bỏ các khoảng im lặng ngắn.
    """
    ste = calculate_ste(y, frame_size, hop_size)
    ste = normalize(ste)
    starts, ends = find_voiced_segments(ste, best_threshold)
    starts, ends = remove_short_silences(starts, ends, hop_size, fs, min_silence_duration)
    return ste, starts, ends

In [15]:
def process_audio_file(folder, file_path, output_folder, frame_size=256, hop_size=128, best_threshold=0.001, min_silence_duration=0.3):
    """
    Xử lý một tệp âm thanh WAV để tìm các phân đoạn nói.
    """
    if folder == 'Sad':
        best_threshold = 0.0003
    y, fs = librosa.load(file_path)
    
    # Tính STE và loại bỏ các khoảng im lặng ngắn
    ste, starts, ends = process_ste(y, fs, frame_size, hop_size, best_threshold, min_silence_duration)
    
    # Nếu có ít nhất một phân đoạn nói được tìm thấy
    if starts and ends:
        # Lấy chỉ số của phân đoạn đầu tiên
        start_index = starts[0]
        end_index = ends[-1]
        
        # Chuyển đổi chỉ số thành thời gian (giây)
        start_time = start_index * hop_size / fs
        end_time = end_index * hop_size / fs
        
        # Hiển thị đồ thị STE và các đoạn phân đoạn nói
        # visualize_ste_with_segments(y, fs, ste, start_index, end_index, best_threshold, file_path)
        
        # Cắt và lưu phân đoạn nói vào thư mục đầu ra
        cut_and_save_segment(file_path, output_folder, start_time, end_time)
    else:
        print("Không tìm thấy phân đoạn nói trong tệp âm thanh.")

In [16]:
def process_folder(folder, input_folder_path,output_folder_path):
    """
    Xử lý tất cả các tệp WAV trong thư mục để tìm các phân đoạn nói.
    """
    for file_name in os.listdir(input_folder_path):
        print(file_name)
        if file_name.endswith('.wav'):
            file_path = os.path.join(input_folder_path, file_name)
            process_audio_file(folder, file_path, output_folder_path)

In [18]:
def process_folders(input_folder, output_folder):
    # Duyệt qua tất cả các thư mục con trong thư mục đầu vào
    for root, dirs, files in os.walk(input_folder):
        print(dirs)
        for folder in dirs:
            # Xác định đường dẫn của thư mục con trong thư mục đầu vào
            subdirectory_input = os.path.join(root, folder)
            
            # Xác định đường dẫn của thư mục con tương ứng trong thư mục đầu ra
            relative_path = os.path.relpath(subdirectory_input, input_folder)
            subdirectory_output = os.path.join(output_folder, relative_path)
            
            # Tạo thư mục con trong thư mục đầu ra nếu nó không tồn tại
            if not os.path.exists(subdirectory_output):
                os.makedirs(subdirectory_output)
            process_folder(folder, subdirectory_input,subdirectory_output)

In [19]:
process_folders('D:/data_analysis/speech_emotion_recognition/data/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/cleaned_data')

['Angry', 'Disgusted', 'Fearful', 'Happy', 'Neutral', 'Sad', 'Suprised']
03-01-05-01-01-01-01.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Angry\03-01-05-01-01-01-01_1.0158730158730158s_to_2.8560544217687074s.wav
1.0158730158730158 2.8560544217687074
03-01-05-01-01-01-02.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Angry\03-01-05-01-01-01-02_1.0216780045351475s_to_2.6528798185941045s.wav
1.0216780045351475 2.6528798185941045
03-01-05-01-01-01-03.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Angry\03-01-05-01-01-01-03_0.9694331065759637s_to_3.0011791383219957s.wav
0.9694331065759637 3.0011791383219957
03-01-05-01-01-01-04.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Angry\03-01-05-01-01-01-04_1.0100680272108844s_to_2.670294784580499s.wav
1.0100680272108844 2.670294784580499
03-01-05-01-01-01-05.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Angry\03-01-05-01-01-01-05_0.9984580498866213s

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10108\1789230656.py:11: RuntimeWarning: invalid value encountered in divide
  return ste / np.max(ste)


0.005804988662131519 3.5526530612244898
1080_ITH_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_ITS_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_IWL_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_IWW_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_MTI_SAD_XX.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Sad\1080_MTI_SAD_XX_0.0s_to_2.9373242630385485s.wav
0.0 2.9373242630385485
1080_TAI_SAD_XX.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Sad\1080_TAI_SAD_XX_0.0s_to_0.5979138321995465s.wav
0.0 0.5979138321995465
1080_TIE_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_TSI_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1080_WSI_SAD_XX.wav
D:/data_analysis/speech_emotion_recognition/data/cleaned_data\Sad\1080_WSI_SAD_XX_0.0s_to_3.2914285714285714s.wav
0.0 3.2914285714285714
1081_DFA_SAD_XX.wav
Không tìm thấy phân đoạn nói trong tệp âm thanh.
1081_I

In [20]:
from scipy.io import wavfile
from scipy.signal import wiener
import soundfile as sf

# Đọc file âm thanh
data, samplerate = sf.read(r'D:\data_analysis\test3\data\17.26, 24 thg 4_.wav')

# Áp dụng bộ lọc Wiener
filtered_data = wiener(data)

# Ghi dữ liệu đã được lọc vào file mới
sf.write('output.wav', filtered_data, samplerate)

d:\data_analysis\speech_emotion_recognition\.venv\Lib\site-packages\scipy\signal\_signaltools.py:1659: RuntimeWarning: divide by zero encountered in divide
  res *= (1 - noise / lVar)
d:\data_analysis\speech_emotion_recognition\.venv\Lib\site-packages\scipy\signal\_signaltools.py:1659: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)
